# Project: Garden Simulator Text Based Game
This project incorporates object-oriented programming concepts like classes and inheritance to represent various plants and the gardener.

In [15]:
import random

In [2]:
class Plant():
    def __init__(self):
        self.growth_stages = ["seed", "sprout", "plant", "flower", "harvest_ready"]
        self.current_growth_stage = self.growth_stages[0]
        self.harvestable = False
        
    def grow(self):
        current_index = self.growth_stages.index(self.current_growth_stage)
        if current_index <= len(self.growth_stages) - 2:
            new_index = current_index + 1
            self.current_growth_stage = self.growth_stages[new_index]
            if new_index == len(self.growth_stages) - 1:
                self.harvestable = True
                print(f"You can harvest the plant {self.name}!")
        elif self.harvestable:
            print(f"{self.name} cannot grow any more.")
            
    def harvest(self):
        if self.harvestable:
            self.current_growth_stage = self.growth_stages[0]
            self.harvestable = False
            return True
        else:
            print("The plant is not ready yet.")

In [3]:
class Tomato(Plant):
    def __init__(self):
        super().__init__()
        self.name = 'Tomato'
        self.growth_stages = ["seed", "sprout", "plant", "flower", "fruiting", "harvest_ready"]

In [4]:
class Carrot(Plant):
    def __init__(self):
        super().__init__()
        self.name = 'Carrot'

In [5]:
class Apple(Plant):
    def __init__(self):
        super().__init__()
        self.name = 'Apple'
        self.growth_stages = ["seed", "sprout", "plant", "flower", "fruiting", "harvest_ready"]

In [6]:
class Lettuce(Plant):
    def __init__(self):
        super().__init__()
        self.name = 'Lettuce'
        
    def fertilize(self):
        current_index = self.growth_stages.index(self.current_growth_stage)
        if current_index <= len(self.growth_stages) - 2:
            self.current_growth_stage = self.growth_stages[len(self.growth_stages) - 1]
            print("Your lettuce is magically available for harvest!")
            self.harvestable = True
        else:
            print("The lettuce is too mature to fertilize.")

In [11]:
class Gardener:
    def __init__(self, name):
        '''name represents the player's name, 
        planted_plants is a list of the plants that are currently planted, 
        and inventory is a dictionary that stores the gardener's collection of seeds 
        and harvested plants.'''
        # Tis needs updating as more fruits come
        self.plant_dict = {'Tomato': Tomato, 'Carrot': Carrot, 'Lettuce': Lettuce, 'Apple': Apple}
        self.name = name.capitalize()
        self.planted_plants = []
        self.inventory = {}
        self.fertilizer_storage = {}

    def plant(self):
        plant_dict = self.plant_dict
        
        if len(self.inventory) < 1:
            print(', but your inventory is empty!')
            return
        chosen_plant = select_items(self.inventory)
        
        if chosen_plant in self.inventory and self.inventory[chosen_plant] > 0:
            self.inventory[chosen_plant] -= 1 
            self.planted_plants.append(self.plant_dict[chosen_plant]()) # plant the instant
            print("You planted a", chosen_plant, " !")
            # Delete plant if zero
            if self.inventory[chosen_plant] < 1:
                del self.inventory[chosen_plant]
                
    def eat(self):
        plant_dict = self.plant_dict
        
        if len(self.inventory) < 1:
            print(', but your inventory is empty!')
            return
        chosen_plant = select_items(self.inventory)
        
        if chosen_plant in self.inventory and self.inventory[chosen_plant] > 0:
            self.inventory[chosen_plant] -= 1 
            print("You ate a", chosen_plant, " ! Delicious.")
            # Delete plant if zero
            if self.inventory[chosen_plant] < 1:
                del self.inventory[chosen_plant]


    def tend(self):
        if len(self.planted_plants) < 1:
            print('No planted plants - empty fields!')
            return
        for plant in self.planted_plants:
            if plant.harvestable:
                print(f'{self.name}, your {plant.name} is harvestable.')
                continue
            else:
                if type(plant) == Lettuce and self.fertilizer_storage.get('Fertilizer', False):
                    answer = input('Type "Yes" if you want to fertilize your lettuce and accelerate the growth! It will consume your fertilizer.')
                    if answer == 'Yes':
                        self.fertilizer_storage['Fertilizer'] -= 1
                        plant.fertilize()
                        if self.fertilizer_storage['Fertilizer'] < 1:
                            del self.fertilizer_storage['Fertilizer']
                    else:
                        print('You did not use the fertilizer. It will grow as usual.')
                        plant.grow()
                else:      
                    plant.grow()
                print(f"Your plant, {plant.name} has grown and is now in stage {plant.current_growth_stage}.")

    def harvest(self):
        if len(self.planted_plants) < 1:
            print('No planted plants - empty fields!')
            return
        chosen_plant = select_items(self.planted_plants) # a plant object
        if chosen_plant.harvestable: 
            self.planted_plants.remove(chosen_plant)
            harvested_n = random.choice([1, 2, 3])
            if chosen_plant in self.inventory:
                self.inventory[chosen_plant.name] += harvested_n
                print(f"You harvested {harvested_n} x {chosen_plant.name}!")
            else:
                self.inventory[chosen_plant.name] = harvested_n
                print(f"You harvested {harvested_n} x {chosen_plant.name}!")
        else:
            print(f"Oops, you still cannot harvest {chosen_plant.name}!")
            
    def forage_for_seeds(self):
        import random
        plants = list(self.plant_dict.keys())
        plants.append('Fertilizer')
        random_plant = random.choice(plants)
        if random_plant == 'Fertilizer':
            if random_plant in self.inventory:
                self.fertilizer_storage[random_plant] += 1
            else:
                self.fertilizer_storage[random_plant] = 1
        else:
            if random_plant in self.inventory:
                self.inventory[random_plant] += 1
            else:
                self.inventory[random_plant] = 1
        print('You just found 1x', random_plant)
        
    def _reset(self):
        answer = input('Reset? It will clear all your data. Type "Yes".')
        if answer == "Yes":
            self.planted_plants = []
            self.inventory = {}
        else:
            pass


In [12]:
def select_items(items):
    ''' a helper function for players to choose an item from their inventory'''
    while True:
        if type(items) == dict:
            for i, key in enumerate(items):
                print(f"{i+1}: {key} x {items.get(key)}")
        elif type(items) == list:
            for i, item in enumerate(items):
                if isinstance(item, Plant):
                    print(f"{i+1}: {item.name};")
                else:
                    print(f"{i+1}: {item};")
        else:
            print("Error: Provided argument is neither a dictionary nor a list.")
            return None
    
    
        x = input('Which do you want to select? Enter a number. ')
        try:
            x = int(x)
            clear_output(wait=True)
            if x > len(items) or x <= 0:
                raise ValueError('Your number is out of range.')
                
            if type(items) == dict:
                for i, key in enumerate(items.keys()):
                    if i == x - 1:
                        return key
            elif type(items) == list:
                return items[x - 1]

        except ValueError:
            clear_output(wait=True)
            print('You should enter a valid number!')
            continue 
        

In [13]:
from IPython.display import clear_output
name = input('What is your name? ')
while len(name) > 20 or len(name) == 0:
    clear_output(wait=True)
    print("Please enter a valid name!")
    clear_output(wait=True)
    name = input('What is your name? ')

    

What is your name? jacob


In [17]:
gardener = Gardener(name)    
list_valid_actions = ['Plant', 'Eat', 'Tend', 'Harvest', 'Forage for Seeds']

# Welcoming message
print(f'Hello, {gardener.name}! Welcome. In this world you can choose to {", ".join(list_valid_actions)}! \n')

while True:
    selected_action = select_items(list_valid_actions).lower().replace(' ', '_')
    if hasattr(gardener, selected_action): # if attr exists

        print(f'You chose to {selected_action.replace("_", " ")}')
        action = getattr(gardener, selected_action)
        action()

        further_action = select_items(['Continue', 'Help', 'Exit'])
        if further_action == 'Continue':
            continue
        elif further_action == 'Help':
            print('You should first forage for seeds and then plant them! You can grow them.')
        elif further_action == 'Exit':
            print('You are exiting the game.')
            gardener._reset()
            break

    else:
        print('Invalid action... Hmm...')

You are exiting the game.
Reset? It will clear all your data. Type "Yes".Yes


##### 